<a href="https://colab.research.google.com/github/MaikarfiJesse/chatBot/blob/main/ClimateChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import BertTokenizer, BertModel
from sklearn.preprocessing import LabelBinarizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Import BERT-base pretrained model
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
import numpy as np
import pandas as pd
import re
import random
import transformers
import matplotlib.pyplot as plt
import json
import pickle
import torch

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import metrics

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
climate_df = pd.read_csv("/content/sample_data/climate.csv")
climate_df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/climate.csv'

In [ ]:
dict_ = {"intents":[]}

# dict_
questions = None
for ind in climate_df.index:
    ques = climate_df['Questions'][ind].lower()
    ques = ques.replace('?','.')
    # ques += climate_df['Patterns'][ind].lower()
    ques = ques.split('.')
    for i, q in enumerate(ques):
        ques[i] = q.strip()
    ans = climate_df['Answers'][ind]
    dict1 = {'tags': climate_df['Tags'][ind]}
    # dict1['patterns'] = ques[:-1]
    dict1['responses'] = ans
    dict_['intents'].append(dict1)

# print(dict_)
print(dict_["intents"][:2])

with open("climate_change.json", "w") as f:
    f.write(json.dumps(dict_))

[{'tags': 'Definition', 'responses': 'Climate change refers to long-term changes in temperature, precipitation, and other atmospheric conditions on Earth.'}, {'tags': 'Causes', 'responses': 'The main causes of climate change are human activities, such as burning fossil fuels, deforestation, and industrial processes, leading to greenhouse gas emissions.'}]


In [ ]:
words=[]
classes=[]
documents=[]
ignore_letters=['?','!','.',',']


for index, row in climate_df.iterrows():
    question = row['Questions']
    tag = row['Tags']
    # Tokenize the question
    inputs = tokenizer(question, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()

    documents.append((embeddings, tag))
    words.extend(tokenizer.tokenize(question))  # Add tokenized words to the word list

    if tag not in classes:
        classes.append(tag)

lemmatizer=WordNetLemmatizer()

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))

pickle.dump(words, open('cl_words.pkl', 'wb'))
pickle.dump(classes, open('cl_classes.pkl', 'wb'))

# Prepare training data
training = []
output_empty = [0] * len(classes)

for document in documents:
    embeddings, tag = document
    output_row = list(output_empty)
    output_row[classes.index(tag)] = 1
    training.append([embeddings, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = np.vstack(training[:, 0])
train_y = np.array(training[:, 1].tolist())

In [ ]:
from tensorflow.keras.optimizers.schedules import ExponentialDecay

model=Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(train_y[0]),activation='softmax'))


# Define learning rate schedule
lr_schedule = ExponentialDecay(
    initial_learning_rate=0.0001,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True
)

# Use the SGD optimizer with the learning rate schedule
sgd = SGD(learning_rate=lr_schedule, momentum=0.9, nesterov=True)
# sgd=SGD(learning_rate=0.0001,decay=1e-6,momentum=0.9,nesterov=True)


model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy','Precision','Recall'])
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=METRICS)
hist = model.fit(np.array(train_x),np.array(train_y),epochs=50,batch_size=12,verbose=1)
model.save('cl_chatbotmodel.h5', hist)
print('Training Done')

Epoch 1/50
4/4 [==============================] - 2s 20ms/step - loss: 2.0594 - accuracy: 0.1667 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 2/50
4/4 [==============================] - 0s 15ms/step - loss: 1.7860 - accuracy: 0.3958 - precision: 0.4286 - recall: 0.0625   
Epoch 3/50
4/4 [==============================] - 0s 16ms/step - loss: 1.6013 - accuracy: 0.4583 - precision: 1.0000 - recall: 0.1667   
Epoch 4/50
4/4 [==============================] - 0s 12ms/step - loss: 1.5280 - accuracy: 0.3958 - precision: 0.8571 - recall: 0.1250
Epoch 5/50
4/4 [==============================] - 0s 9ms/step - loss: 1.3278 - accuracy: 0.5417 - precision: 0.8333 - recall: 0.2083
Epoch 6/50
4/4 [==============================] - 0s 16ms/step - loss: 1.2697 - accuracy: 0.5833 - precision: 0.8182 - recall: 0.3750
Epoch 7/50
4/4 [==============================] - 0s 13ms/step - loss: 1.2143 - accuracy: 0.5833 - precision: 0.8000 - recall: 0.4167
Epoch 8/50
4/4 [==============================] -

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.models import load_model

with open('climate_change.json') as json_file:
    intents = json.load(json_file)

words=pickle.load(open('cl_words.pkl','rb'))
print(words[:100], len(words))

classes=pickle.load(open('cl_classes.pkl','rb'))
print(classes[:5], len(classes))
model=load_model('cl_chatbotmodel.h5')

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings


def predict_class(sentence):
    embedding = get_bert_embedding(sentence)
    res = model.predict(embedding)[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    print(intents_list[0])
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if tag in i['tags']:
            return i['responses']
    return None

print("GO! BOT IS RUNNING")

while True:
    message=input("")
    if message.lower() in ['quit', 'exit']:
      print("Goodbye")
      break
    ints=predict_class(message)
    if len(ints) > 0:
        res=get_response(ints,intents)
        print(f"Bot: {res}")

['##ence', '##iga', '##ili', '##ing', '##orestation', '##s', '##ting', '##tion', '(', ')', 'activity', 'adapt', 'adaptation', 'addressing', 'affect', 'agreement', 'agriculture', 'and', 'are', 'biodiversity', 'burning', 'can', 'cap', 'capture', 'carbon', 'cause', 'cc', 'change', 'city', 'climate', 'combat', 'contribute', 'coral', 'cycle', 'def', 'do', 'doe', 'economy', 'ecosystem', 'education', 'effect', 'emission', 'energy', 'event', 'example', 'extreme', 'food', 'footprint', 'forest', 'fossil', 'freshwater', 'fuel', 'gas', 'global', 'greenhouse', 'health', 'help', 'how', 'human', 'i', 'ice', 'impact', 'in', 'international', 'is', 'level', 'linked', 'main', 'melting', 'mental', 'mit', 'my', 'ocean', 'of', 'offset', 'on', 'paris', 'pattern', 'polar', 're', 'reduce', 'reef', 'refugee', 'region', 'renewable', 'resource', 'rise', 'role', 'sea', 'security', 'solution', 'some', 'source', 'storage', 'strategy', 'technology', 'the', 'their', 'to', 'transportation'] 105
['Carbon Footprint', 'Ca